# 🤖 Technical Interview AI - Auto-Sync

**Near Real-time Workflow:**
1. ✏️ Edit code in Cursor
2. 🔄 Auto-sync every 30 seconds
3. 🚀 Pull changes in Colab (1 click)
4. 🔥 Train on GPU!

Replace `YOUR_REPO_URL` with your GitHub repository URL.

In [ ]:
# 🔄 QUICK SYNC: Get latest changes from Cursor (30 seconds old max!)
REPO_URL = 'https://github.com/shijazi88/technical-interview-ai'
PROJECT_DIR = 'interview-ai'

import os
if os.path.exists(PROJECT_DIR):
    print("🔄 Pulling latest changes from Cursor...")
    %cd $PROJECT_DIR
    !git pull origin main
    print("✅ Synced! Your latest Cursor code is now here.")
else:
    print("📥 First time: Cloning repository...")
    !git clone $REPO_URL $PROJECT_DIR
    %cd $PROJECT_DIR
    print("✅ Repository cloned!")

!ls -la *.py

In [ ]:
# 📦 Setup environment
!pip install -q transformers peft accelerate bitsandbytes datasets

import torch
print(f"🔥 GPU: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'None - Enable GPU!'}")

In [ ]:
# 🚀 Train with your latest Cursor code!
!python colab_training_pipeline.py --num_scenarios 100 --epochs 3